# AI Agents

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureOpenAI
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
import requests
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import BaseTool, StructuredTool, tool

# Load environment variables from .env file
load_dotenv()

# Access the environment variables
openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_chat_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")


# Initialize the Azure OpenAI model
model = AzureChatOpenAI(
    openai_api_version=openai_api_version,
    azure_deployment=azure_chat_deployment,
)

# Create a human message and invoke the model
message = HumanMessage(
    content="Translate this sentence from English to French. I love programming."
)
response = model.invoke([message])

# Print the response
print(response)


content="J'aime programmer." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 19, 'total_tokens': 24, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_ee1d74bde0', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filt

In [2]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    api_key = openai_api_key,
    azure_deployment="text-embedding-3-large"
)

## SQL Agent

In [3]:
import pandas as pd
import sqlite3
import random

# ----------------------------
# 1. Product data
# ----------------------------
data = {
    "Product ID": [f"P{str(i).zfill(3)}" for i in range(1, 31)],
    "Product Name": [
        "Classic Piadina", "Vegetarian Piadina", "Ham & Cheese Piadina", 
        "Nutella Piadina", "Caprese Salad", "Tigelle Bread", 
        "Sun-Dried Tomatoes", "Parmesan Cheese", "Prosciutto Crudo", 
        "Balsamic Vinegar", "Truffle Piadina", "Spicy Salami Piadina", 
        "Vegan Piadina", "Chicken Caesar Salad", "Garlic Bread", 
        "Olive Tapenade", "Ricotta Cheese", "Mortadella", 
        "Pesto Sauce", "Marinated Artichokes", "Four Cheese Piadina", 
        "Grilled Eggplant", "Smoked Salmon Piadina", "Tuna Salad", 
        "Rosemary Focaccia", "Anchovy Spread", "Gorgonzola Cheese", 
        "Speck", "Fig Jam", "Lemon Olive Oil"
    ],
    "Category": [
        "Piadina", "Piadina", "Piadina", "Dessert", "Salad", "Bread", 
        "Condiment", "Cheese", "Meat", "Condiment", "Piadina", "Piadina", 
        "Piadina", "Salad", "Bread", "Condiment", "Cheese", "Meat", 
        "Condiment", "Condiment", "Piadina", "Vegetable", "Piadina", 
        "Salad", "Bread", "Condiment", "Cheese", "Meat", "Condiment", 
        "Condiment"
    ],
    "Price": [
        5.50, 6.00, 6.50, 4.00, 7.00, 3.50, 4.50, 8.00, 9.00, 6.00, 
        7.50, 6.75, 6.25, 8.50, 3.75, 5.25, 7.75, 8.50, 4.75, 5.50, 
        7.00, 4.25, 9.50, 7.25, 4.00, 5.75, 8.25, 9.25, 6.50, 5.00
    ],
    "Stock": [
        20, 15, 25, 10, 12, 30, 40, 18, 22, 25, 14, 16, 20, 10, 35, 
        28, 15, 12, 30, 25, 18, 22, 14, 10, 40, 20, 15, 12, 18, 25
    ],
    "Allergens": [
        "Gluten, Dairy", "Gluten, Dairy", "Gluten, Dairy, Pork", 
        "Gluten, Dairy, Nuts", "Dairy", "Gluten", "None", "Dairy", 
        "Pork", "None", "Gluten, Dairy, Truffle", "Gluten, Dairy, Pork", 
        "Gluten", "Dairy, Eggs", "Gluten", "None", "Dairy", "Pork", 
        "Nuts", "None", "Gluten, Dairy", "None", "Gluten, Dairy, Fish", 
        "Fish, Eggs", "Gluten", "Fish", "Dairy", "Pork", "None", "None"
    ],
    "Description": [
        "Traditional Italian flatbread filled with prosciutto and cheese.",
        "Flatbread filled with grilled vegetables and mozzarella.",
        "Flatbread filled with ham and melted cheese.",
        "Sweet flatbread filled with Nutella and crushed hazelnuts.",
        "Fresh salad with mozzarella, tomatoes, and basil.",
        "Soft round bread, perfect for pairing with spreads or cured meats.",
        "Rich and tangy sun-dried tomatoes, ideal for salads or toppings.",
        "Aged Parmesan cheese, perfect for grating over dishes.",
        "Thinly sliced cured ham, a classic Italian delicacy.",
        "Authentic balsamic vinegar from Modena, great for salads and marinades.",
        "Flatbread filled with truffle cream and cheese.",
        "Flatbread filled with spicy salami and mozzarella.",
        "Flatbread filled with vegan cheese and grilled vegetables.",
        "Crisp romaine lettuce with grilled chicken and Caesar dressing.",
        "Toasted bread with garlic and olive oil.",
        "Savory olive spread, perfect for appetizers.",
        "Creamy ricotta cheese, ideal for desserts or pasta.",
        "Thinly sliced mortadella, a flavorful Italian sausage.",
        "Fresh basil pesto sauce, great for pasta or sandwiches.",
        "Marinated artichokes, perfect for antipasti platters.",
        "Flatbread filled with a blend of four Italian cheeses.",
        "Grilled eggplant slices, seasoned with olive oil and herbs.",
        "Flatbread filled with smoked salmon and cream cheese.",
        "Fresh tuna salad with celery and mayonnaise.",
        "Soft focaccia bread infused with rosemary.",
        "Savory anchovy spread, ideal for bruschetta.",
        "Creamy Gorgonzola cheese, perfect for sauces or salads.",
        "Smoky cured speck, a northern Italian specialty.",
        "Sweet fig jam, great for pairing with cheese.",
        "Lemon-infused olive oil, perfect for dressings or marinades."
    ],
    "Calories": [
        350, 300, 400, 450, 250, 200, 50, 110, 150, 20, 380, 420, 310, 
        270, 180, 90, 120, 160, 80, 70, 400, 60, 450, 320, 220, 40, 
        130, 170, 100, 30
    ],
    "Vegetarian": [
        False, True, False, False, True, True, True, True, False, True, 
        True, False, True, False, True, True, True, False, True, True, 
        True, True, False, False, True, False, True, False, True, True
    ]
}

# ----------------------------
# 2. Supplier data
# ----------------------------
suppliers = {
    "Supplier ID": [f"S{str(i).zfill(3)}" for i in range(1, 11)],
    "Supplier Name": [
        "PiadaMaster Inc.", "Verde Farms", "Dolce Italia", 
        "Formaggi & Co.", "Salumi Rossi", "Mediterraneo Ltd.", 
        "Truffle Treasures", "Balsamico Tradizionale", 
        "Pane Fresco", "Olio di Sole"
    ],
    "Contact Email": [
        "contact@piadamaster.com", "info@verdefarms.com", 
        "orders@dolceitalia.it", "sales@formaggico.com", 
        "hello@salumirossi.it", "support@mediterraneo.com", 
        "info@truffletreasures.com", "shop@balsamicotradizionale.it", 
        "contact@panefresco.com", "service@oliodisole.it"
    ],
    "Country": [
        "Italy", "Italy", "Italy", "Italy", "Italy", 
        "Greece", "Italy", "Italy", "Italy", "Spain"
    ]
}

# ----------------------------
# 3. Create DataFrames
# ----------------------------
products_df = pd.DataFrame(data)
suppliers_df = pd.DataFrame(suppliers)

# ----------------------------
# 4. Assign a random Supplier ID to each product
# ----------------------------
supplier_ids = suppliers_df["Supplier ID"].tolist()
products_df["Supplier ID"] = [random.choice(supplier_ids) for _ in range(len(products_df))]

# ----------------------------
# 5. Save to SQLite
# ----------------------------
conn = sqlite3.connect("piadineria.db")

products_df.to_sql("products", conn, if_exists="replace", index=False)
suppliers_df.to_sql("suppliers", conn, if_exists="replace", index=False)

conn.close()


In [3]:
from langchain_community.utilities.sql_database import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///piadineria.db")

from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

sql_toolkit = SQLDatabaseToolkit(db=db, llm=model)
sql_toolkit.get_tools()

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000256933169F0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000256933169F0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000256933169F0>),
 QuerySQLCheckerTool(description='Use this tool to 

## Add to cart tool

In [4]:
# Define the tool to add an item to the cart
@tool
def add_to_cart(item_name: str, item_price: float) -> str:
    """Add an item to the cart."""
    url = 'http://localhost:3000/cart'  # Ensure this matches the JSON Server endpoint
    cart_item = {
        'name': item_name,
        'price': item_price
    }
    
    response = requests.post(url, json=cart_item)
    
    if response.status_code == 201:
        return f"Item '{item_name}' added to cart successfully."
    else:
        return f"Failed to add item to cart: {response.status_code} {response.text}"


# Retriever tool

In [11]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
import time
from langchain_text_splitters import CharacterTextSplitter

from langchain.document_loaders import PyPDFDirectoryLoader


index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

file_path = (
    "documents"
)
loader = PyPDFDirectoryLoader(file_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [12]:
from langchain_community.retrievers import AzureAISearchRetriever
vector_store.add_documents(documents=docs)

retriever = vector_store.as_retriever()

from langchain.tools.retriever import create_retriever_tool

rag_tool = create_retriever_tool(
    retriever,
    "document_search",
    """
    Search and return information restaurants health certificate and owner's history.
    """
)

## Prompt Design

In [ ]:
# Define the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are an AI assistant for a Piadineria Restaurant. 
            You help customers explore the menu and choose the best piadine or Italian specialties through friendly, interactive questions.
            When the user asks for product details (ingredients, allergens, vegetarian options, price, etc.), you can query the product database.

            Once the user is ready to order, ask if they'd like to add the selected item to their cart.
            If they confirm, add the item to the cart using your tools.

            When using a tool, respond only with the final result. For example:
            Human: Add Classic Piadina to the cart with price 5.50
            AI: Item 'Classic Piadina' added to cart successfully.
        """),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

## Running the Agent

In [17]:
import os
load_dotenv()
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = "askmamma"



In [20]:
from langchain.callbacks.tracers import LangChainTracer
from langsmith import Client

client = Client(
  api_key=os.getenv("LANGSMITH_API_KEY"),  # This can be retrieved from a secrets manager
  api_url=os.getenv("LANGSMITH_ENDPOINT"),  # Update appropriately for self-hosted installations or the EU region
)

tracer = LangChainTracer(client=client, project_name="askmamma")


In [23]:
# Setup the toolkit
toolkit = [rag_tool, add_to_cart, sql_toolkit.get_tools()[0], sql_toolkit.get_tools()[1], sql_toolkit.get_tools()[2], sql_toolkit.get_tools()[3]]

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
 
message_history = ChatMessageHistory()

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(model, toolkit, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=toolkit, verbose=True)

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)
 


# Interactive loop
while True:
    user_input = input("Type your question here (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break
    result = agent_with_chat_history.invoke(
        {"input": user_input},
        # This is needed because in most real world scenarios, a session id is needed
        # It isn't really used here because we are using a simple in memory ChatMessageHistory
        config={"configurable": {"session_id": "<foo>"}, "callbacks": [tracer]},
    )
    print(f'User: {user_input}')
    print(f'AI: {result['output']}')



> Entering new AgentExecutor chain...
Hello! Welcome to our Piadineria Restaurant. How can I assist you today? Are you looking to explore our menu or do you have any specific questions?

> Finished chain.
User: hi
AI: Hello! Welcome to our Piadineria Restaurant. How can I assist you today? Are you looking to explore our menu or do you have any specific questions?


# Agent output evaluation

In [ ]:
from langsmith import Client

client = Client()

# Define dataset: these are your test cases
dataset_name = "QA Example Dataset"
dataset = client.create_dataset(dataset_name)
client.create_examples(
    dataset_id=dataset.id,
    examples=[
        {
            "inputs": {"question": "What are the ingredients in the Classic Piadina?"},
            "outputs": {"answer": "Traditional Italian flatbread filled with prosciutto and cheese."},
        },
        {
            "inputs": {"question": "Is the Vegetarian Piadina gluten-free?"},
            "outputs": {"answer": "No, it contains gluten and dairy"},
        },
        {
            "inputs": {"question": "Add Classic Piadina to the cart with price 5.50."},
            "outputs": {"answer": "Item 'Classic Piadina' added to cart successfully."},
        }
    ]
)

# Agent tool evaluation

In [ ]:
import pytest
from langsmith import testing as t

@pytest.mark.langsmith
def test_searches_for_correct_ticker() -> None:
  """Test that the model looks up the correct ticker on simple query."""
  # Log the test example
  query = "What is the price of Apple?"
  t.log_inputs({"query": query})
  expected = "AAPL"
  t.log_reference_outputs({"ticker": expected})

  # Call the agent's model node directly instead of running the full ReACT loop.
  result = agent.nodes["agent"].invoke(
      {"messages": [{"role": "user", "content": query}]}
  )
  tool_calls = result["messages"][0].tool_calls
  if tool_calls[0]["name"] == polygon_aggregates.name:
      actual = tool_calls[0]["args"]["ticker"]
  else:
      actual = None
  t.log_outputs({"ticker": actual})

  # Check that the right ticker was queried
  assert actual == expected